<a href="https://colab.research.google.com/github/anshmehta337/PyTorch/blob/master/t4smalldailydialog.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install transformers datasets accelerate

In [ ]:
pip install --upgrade datasets


In [ ]:
from google.colab import files

uploaded = files.upload()



Saving train.csv to train.csv


In [ ]:
import pandas as pd
dataset=pd.read_csv('train.csv')

In [ ]:
dataset

,dialog,act,emotion
0,"['Say , Jim , how about going for a few beers ...",[3 4 2 2 2 3 4 1 3 4],[0 0 0 0 0 0 4 4 4 4]
1,"['Can you do push-ups ? '\n "" Of course I can ...",[2 1 2 2 1 1],[0 0 6 0 0 0]
2,"['Can you study with the radio on ? '\n ' No ,...",[2 1 2 1 1],[0 0 0 0 0]
3,['Are you all right ? '\n ' I will be all righ...,[2 1 1 1],[0 0 0 0]
4,"['Hey John , nice skates . Are they new ? '\n ...",[2 1 2 1 1 2 1 3 4],[0 0 0 0 0 6 0 6 0]
...,...,...,...
11113,"['Hello , I bought a pen in your shop just bef...",[1 1 1 2 3 2 1 4 1],[0 4 0 0 0 0 0 0 4]
11114,['Do you have any seats available ? ' ' Yes . ...,[2 1 2 1 3 4],[0 0 0 0 0 4]
11115,"['Uncle Ben , how did the Forbidden City get t...",[2 1 2 1 1 1 1 1 2 1 2 1 2 1 3 4],[0 0 6 0 6 0 0 0 0 0 0 0 0 0 4 0]
11116,"['May I help you , sir ? ' ' I want a pair of ...",[2 3 4 3],[0 0 0 0]


In [ ]:
#we will use ast.literal_eval to find out string correctly

In [ ]:
import ast

In [ ]:
dataset['dialog']=dataset['dialog'].apply(ast.literal_eval)

In [ ]:
dataset['dialog']

,dialog
0,"[Say , Jim , how about going for a few beers a..."
1,[Can you do push-ups ? Of course I can . It's...
2,"[Can you study with the radio on ? No , I lis..."
3,[Are you all right ? I will be all right soon...
4,"[Hey John , nice skates . Are they new ? Yeah..."
...,...
11113,"[Hello , I bought a pen in your shop just befo..."
11114,[Do you have any seats available ? Yes . This...
11115,"[Uncle Ben , how did the Forbidden City get th..."
11116,"[May I help you , sir ? I want a pair of locu..."


In [ ]:
store=[]
for i in dataset['dialog']:
  for j in range (len(i)-1):
    inp=str(i[j]).strip()
    out=str(i[j+1]).strip()
    store.append((inp,out))

In [ ]:
print(len(store))

0


In [ ]:
import ast

dataset['dialog'] = dataset['dialog'].apply(lambda x: ast.literal_eval(x) if isinstance(x, str) else x)
print(dataset['dialog'].iloc[0])
print(type(dataset['dialog'].iloc[0]))



["Say , Jim , how about going for a few beers after dinner ?  You know that is tempting but is really not good for our fitness .  What do you mean ? It will help us to relax .  Do you really think so ? I don't . It will just make us fat and act silly . Remember last time ?  I guess you are right.But what shall we do ? I don't feel like sitting at home .  I suggest a walk over to the gym where we can play singsong and meet some of our friends .  That's a good idea . I hear Mary and Sally often go there to play pingpong.Perhaps we can make a foursome with them .  Sounds great to me ! If they are willing , we could ask them to go dancing with us.That is excellent exercise and fun , too .  Good.Let ' s go now .  All right . "]
<class 'list'>


In [ ]:
store = []

for conv in dataset['dialog']:


        text = conv[0]

        turns = [t.strip() for t in text.split("  ") if t.strip()]

        for i in range(len(turns) - 1):
            store.append((turns[i], turns[i + 1]))

print("Total pairs:", len(store))
print("Example:", store[0])


Total pairs: 76052
Example: ('Say , Jim , how about going for a few beers after dinner ?', 'You know that is tempting but is really not good for our fitness .')


In [ ]:
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM

model_name = "t5-small"
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForSeq2SeqLM.from_pretrained(model_name)


/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/2.32k [00:00<?, ?B/s]

spiece.model:   0%|          | 0.00/792k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.39M [00:00<?, ?B/s]

config.json:   0%|          | 0.00/1.21k [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/242M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/147 [00:00<?, ?B/s]

In [ ]:
from torch.utils.data import Dataset

class DialogueDataset(Dataset):
    def __init__(self, pairs, tokenizer, max_len=128):
        self.pairs = pairs
        self.tokenizer = tokenizer
        self.max_len = max_len

    def __len__(self):
        return len(self.pairs)

    def __getitem__(self, idx):
        src, tgt = self.pairs[idx]
        inputs = self.tokenizer(src, truncation=True, padding="max_length", max_length=self.max_len, return_tensors="pt")
        targets = self.tokenizer(tgt, truncation=True, padding="max_length", max_length=self.max_len, return_tensors="pt")
        return {
            "input_ids": inputs["input_ids"].squeeze(),
            "attention_mask": inputs["attention_mask"].squeeze(),
            "labels": targets["input_ids"].squeeze(),
        }


In [ ]:
from torch.utils.data import DataLoader
from sklearn.model_selection import train_test_split

train_data, test_data = train_test_split(store, test_size=0.1, random_state=42)
print(f"Train: {len(train_data)}, Test: {len(test_data)}")

train_dataset = DialogueDataset(train_data, tokenizer)
test_dataset = DialogueDataset(test_data, tokenizer)

train_loader = DataLoader(train_dataset, batch_size=16, shuffle=True)



Train: 68446, Test: 7606


In [ ]:
import os
os.environ["WANDB_MODE"] = "disabled"


In [ ]:
from transformers import Trainer, TrainingArguments

training_args = TrainingArguments(
    output_dir="./results",
    eval_strategy="no",
    save_strategy="epoch",
    learning_rate=5e-5,
    per_device_train_batch_size=8,
    num_train_epochs=3,
    weight_decay=0.01,
    logging_dir="./logs",
    logging_steps=50,
)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
)

trainer.train()


<IPython.core.display.Javascript object>

wandb: Logging into wandb.ai. (Learn how to deploy a W&B server locally: https://wandb.me/wandb-server)
wandb: You can find your API key in your browser here: https://wandb.ai/authorize?ref=models
wandb: Paste an API key from your profile and hit enter:wandb: WARNING If you're specifying your api key in code, ensure this code is not shared publicly.
wandb: WARNING Consider setting the WANDB_API_KEY environment variable, or running `wandb login` from the command line.
wandb: No netrc file found, creating one.
wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc
wandb: Currently logged in as: ansh138095 (ansh138095-netaji-subhas-university-of-technology) to https://api.wandb.ai. Use `wandb login --relogin` to force relogin


Step,Training Loss
50,5.313800
100,1.026100
150,0.662200
200,0.590800
250,0.511100
300,0.488500
350,0.498600
400,0.470900
450,0.466400
500,0.466200


Step,Training Loss
50,5.313800
100,1.026100
150,0.662200
200,0.590800
250,0.511100
300,0.488500
350,0.498600
400,0.470900
450,0.466400
500,0.466200


TrainOutput(global_step=25668, training_loss=0.42735417080163324, metrics={'train_runtime': 4198.8813, 'train_samples_per_second': 48.903, 'train_steps_per_second': 6.113, 'total_flos': 6947703707664384.0, 'train_loss': 0.42735417080163324, 'epoch': 3.0})

In [ ]:
model.save_pretrained("./paraphrase_model")
tokenizer.save_pretrained("./paraphrase_model")


('./paraphrase_model/tokenizer_config.json',
 './paraphrase_model/special_tokens_map.json',
 './paraphrase_model/spiece.model',
 './paraphrase_model/added_tokens.json',
 './paraphrase_model/tokenizer.json')

In [ ]:
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM

tokenizer = AutoTokenizer.from_pretrained("./paraphrase_model")
model = AutoModelForSeq2SeqLM.from_pretrained("./paraphrase_model")


In [ ]:
from transformers import AutoTokenizer, AutoModelForCausalLM

# Load a model trained for text continuation (causal LM)
model_name = "gpt2"  # or "gpt2-medium" / "gpt2-large" for better results
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForCausalLM.from_pretrained(model_name)

def generate_next_sentence(prompt, max_new_tokens=30, temperature=0.8, top_p=0.95):
    inputs = tokenizer(prompt, return_tensors="pt")
    outputs = model.generate(
        **inputs,
        max_new_tokens=max_new_tokens,
        temperature=temperature,
        top_p=top_p,
        do_sample=True,
        pad_token_id=tokenizer.eos_token_id
    )
    generated_text = tokenizer.decode(outputs[0], skip_special_tokens=True)

    # Only show the continuation part
    continuation = generated_text[len(prompt):].strip()
    return continuation

# Example:
text = "He's playing football."
print(generate_next_sentence(text))


tokenizer_config.json:   0%|          | 0.00/26.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/665 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/1.04M [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/548M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/124 [00:00<?, ?B/s]

It's not an easy thing to do, but he's an outstanding player. We want to bring him back."

The Saints took their time


In [ ]:
text = "have you done dsa?"
print(generate_next_sentence(text))


KARL (CONT'D) You see, I have had very little trouble getting you to join the guild. I was just thinking


In [ ]:
!zip -r paraphrase_model.zip paraphrase_model
from google.colab import files
files.download("paraphrase_model.zip")


  adding: paraphrase_model/ (stored 0%)
  adding: paraphrase_model/special_tokens_map.json (deflated 85%)
  adding: paraphrase_model/generation_config.json (deflated 29%)
  adding: paraphrase_model/model.safetensors (deflated 8%)
  adding: paraphrase_model/tokenizer.json (deflated 74%)
  adding: paraphrase_model/spiece.model (deflated 48%)
  adding: paraphrase_model/tokenizer_config.json (deflated 95%)
  adding: paraphrase_model/config.json (deflated 63%)


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>